In [54]:
import numpy as np 

import os
from os import listdir

os.environ["KERAS__BACKEND"] = "tensorflow"

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Flatten, Dense, MaxPooling2D, Dropout, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils import shuffle
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score ,accuracy_score
import imutils

In [55]:
image_dir="./archive/brain_tumor_dataset/"

In [56]:
def augment_data(file_dir, n_generated_samples, save_to_dir, augment_params=None, verbose=False):
    augment_params = augment_params or {
        'rotation_range': 10,
        'width_shift_range': 0.1,
        'height_shift_range': 0.1,
        'shear_range': 0.1,
        'brightness_range': (0.3, 1.0),
        'horizontal_flip': True,
        'vertical_flip': True,
        'fill_mode': 'nearest'
    }
    
    data_gen = ImageDataGenerator(**augment_params)

    os.makedirs(save_to_dir, exist_ok=True)

    for filename in os.listdir(file_dir):
        file_path = os.path.join(file_dir, filename)
        
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            if verbose:
                print(f"Skipping non-image file: {filename}")
            continue
        
        try:
            image = cv2.imread(file_path)
            if image is None:
                if verbose:
                    print(f"Failed to load image: {filename}")
                continue

            image = image.reshape((1,) + image.shape)
            save_prefix = 'aug_' + os.path.splitext(filename)[0]

            for i, batch in enumerate(data_gen.flow(x=image, batch_size=1, 
                                                     save_to_dir=save_to_dir, 
                                                     save_prefix=save_prefix, 
                                                     save_format='jpg')):
                if i >= n_generated_samples:
                    break
            
            if verbose:
                print(f"Augmented {filename} -> {n_generated_samples} samples")
        
        except Exception as e:
            if verbose:
                print(f"Error processing {filename}: {e}")


In [57]:
os.makedirs('augmented-images')
os.makedirs('augmented-images/yes')
os.makedirs('augmented-images/no')
augmented_data_path ='./augmented-images/'

In [58]:
from concurrent.futures import ThreadPoolExecutor

tasks = [
    {"file_dir": image_dir + "yes", "n_generated_samples": 6, "save_to_dir": augmented_data_path + "yes"},
    {"file_dir": image_dir + "no", "n_generated_samples": 9, "save_to_dir": augmented_data_path + "no"},
]

def run_augmentation(task):
    augment_data(
        file_dir=task["file_dir"],
        n_generated_samples=task["n_generated_samples"],
        save_to_dir=task["save_to_dir"],
        verbose=True
    )


with ThreadPoolExecutor() as executor:
    futures = [executor.submit(run_augmentation, task) for task in tasks]
    for future in futures:
        future.result()  

Augmented Y105.jpg -> 6 samples
Augmented no 90.jpg -> 9 samples
Augmented Y154.jpg -> 6 samples
Augmented Y75.JPG -> 6 samples
Augmented Y61.jpg -> 6 samples
Augmented no 94.jpg -> 9 samples
Augmented 36 no.jpg -> 9 samples
Augmented No18.jpg -> 9 samples
Augmented 11 no.jpg -> 9 samples
Augmented Y257.jpg -> 6 samples
Augmented Y180.jpg -> 6 samples
Augmented Y8.jpg -> 6 samples
Augmented no 96.jpg -> 9 samples
Augmented Y38.jpg -> 6 samples
Augmented 37 no.jpg -> 9 samples
Augmented Y194.jpg -> 6 samples
Augmented 20 no.jpg -> 9 samples
Augmented Y147.JPG -> 6 samples
Augmented Y92.jpg -> 6 samples
Augmented Y255.JPG -> 6 samples
Augmented Y254.jpg -> 6 samples
Augmented Y9.jpg -> 6 samples
Augmented no 98.jpg -> 9 samples
Augmented N15.jpg -> 9 samples
Augmented No22.jpg -> 9 samples
Augmented 41 no.jpg -> 9 samples
Augmented 19 no.jpg -> 9 samples
Augmented 8 no.jpg -> 9 samples
Augmented No16.jpg -> 9 samples
Augmented 35 no.jpg -> 9 samples
Augmented 27 no.jpg -> 9 samples
Augme

In [59]:
def crop_brain_contour(image, blur_kernel=(5, 5), threshold=45, plot=False):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    gray = cv2.GaussianBlur(gray, blur_kernel, 0)

    thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)[1]

    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    if not cnts:
        raise ValueError("No contours found. Check the input image or thresholding parameters.")

    c = max(cnts, key=cv2.contourArea)

    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # Crop the image using the extreme points
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]

    if plot:
        import matplotlib.pyplot as plt
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.title('Original Image')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)
        plt.title('Cropped Image')
        plt.axis('off')
        plt.show()

    return new_image


In [60]:
import cv2
import numpy as np
from os import listdir
from sklearn.utils import shuffle
from concurrent.futures import ThreadPoolExecutor

def load_data(dir_list, image_size, crop_func=crop_brain_contour, progress_bar=False):
    
    image_width, image_height = image_size
    
    def process_image(directory, filename):
        try:
            image_path = f"{directory}/{filename}"
            image = cv2.imread(image_path)

            if image is None:
                print(f"Warning: Failed to load image {filename}")
                return None, None

            if crop_func:
                image = crop_func(image)

            image = cv2.resize(image, (image_width, image_height), interpolation=cv2.INTER_CUBIC)
            image = image / 255.0

            label = [1] if 'yes' in directory else [0]
            
            return image, label
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            return None, None

    X, y = [], []

    with ThreadPoolExecutor() as executor:
        futures = []
        
        for directory in dir_list:
            for filename in listdir(directory):
                futures.append(executor.submit(process_image, directory, filename))
        

        for future in futures:
            image, label = future.result()
            if image is not None:
                X.append(image)
                y.append(label)
    
    X = np.array(X)
    y = np.array(y)

    # Shuffle data
    X, y = shuffle(X, y)

    # Print dataset information
    print(f'Number of examples: {len(X)}')
    print(f'X shape: {X.shape}')
    print(f'y shape: {y.shape}')

    return X, y


In [61]:
augmented_yes =augmented_data_path+'yes'
augmented_no = augmented_data_path+'no'

IMG_WIDTH, IMG_HEIGHT = (240, 240)

X, y = load_data([augmented_yes, augmented_no], (IMG_WIDTH, IMG_HEIGHT))

Number of examples: 2065
X shape: (2065, 240, 240, 3)
y shape: (2065, 1)


In [62]:
def split_data(X, y, test_size=0.2):
       
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=test_size)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [63]:
X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y, test_size=0.3)

In [71]:
def cnn_model(input_shape):
    activation_function = 'relu'
    pool_size = (2, 2)
    kernel_size = (3, 3)
    
    def conv_block(inputs, filters, activation=activation_function, pool_size=pool_size, kernel_size=kernel_size):
        """Helper function to create a convolutional block with Conv2D, MaxPooling, and BatchNormalization."""
        x = Conv2D(filters, kernel_size, activation=activation, padding='same')(inputs)
        x = MaxPooling2D(pool_size)(x)
        x = BatchNormalization(axis=3)(x)
        return x

    inputs = Input(shape=input_shape)
    
    x = conv_block(inputs, 32)
    x = conv_block(x, 32)
    x = conv_block(x, 64)
    x = conv_block(x, 128)
    x = conv_block(x, 256)
    x = conv_block(x, 512)
    
    x = Flatten()(x)
    
    x = Dense(256, activation=activation_function)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    x = Dense(128, activation=activation_function)(x)
    x = BatchNormalization()(x)
    
    output = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=output)
    
    return model


In [72]:
IMG_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
model=cnn_model(IMG_SHAPE)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 240, 240, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 240, 240, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 120, 120, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 120, 120, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 120, 120, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 60, 60, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 60, 60, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 60, 60, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 30, 30, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_27 (MaxPooling2D) │ (None, 15, 15, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 15, 15, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 15, 15, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_28 (MaxPooling2D) │ (None, 7, 7, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 7, 7, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 7, 7, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_29 (MaxPooling2D) │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 3, 3, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 256)            │         1,02

 Total params: 2,796,385 (10.67 MB)

 Trainable params: 2,793,569 (10.66 MB)

 Non-trainable params: 2,816 (11.00 KB)

In [73]:
checkpoint_file = 'best_model2.keras'


checkpoint_loss = ModelCheckpoint(
    'best_loss_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)


checkpoint_acc = ModelCheckpoint(
    'best_accuracy_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

reduce_lr1 = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1)

In [74]:
model.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])

In [75]:
model.fit(x=X_train, y=y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val),
                callbacks=[checkpoint_loss, checkpoint_acc, reduce_lr1])

Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.7220 - loss: 0.5993

2024-12-23 02:24:29.731759: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion', 76 bytes spill stores, 76 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_5', 68 bytes spill stores, 68 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_reduce_window_fusion_5', 76 bytes spill stores, 76 bytes spill loads




Epoch 1: val_loss improved from inf to 1.72924, saving model to best_loss_model.keras

Epoch 1: val_accuracy improved from -inf to 0.44839, saving model to best_accuracy_model.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 14s 176ms/step - accuracy: 0.7233 - loss: 0.5968 - val_accuracy: 0.4484 - val_loss: 1.7292 - learning_rate: 0.0010
Epoch 2/50
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.8500 - loss: 0.3425
Epoch 2: val_loss did not improve from 1.72924

Epoch 2: val_accuracy did not improve from 0.44839
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 70ms/step - accuracy: 0.8515 - loss: 0.3403 - val_accuracy: 0.4484 - val_loss: 1.7648 - learning_rate: 0.0010
Epoch 3/50
45/46 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9357 - loss: 0.1801
Epoch 3: val_loss improved from 1.72924 to 1.33364, saving model to best_loss_model.keras

Epoch 3: val_accuracy improved from 0.44839 to 0.49032, saving model to best_accuracy_model.keras
46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 90ms/step - accuracy: 0.9356 - loss: 0.1797 - v

In [76]:
loss, accuracy = model.evaluate(X_test, y_test)

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Test loss: {loss * 100:.2f}%")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.9836 - loss: 0.0636
Test Accuracy: 97.42%
Test loss: 9.14%


In [77]:
pred = model.predict(X_test)

predicted_classes = (pred > 0.5).astype("int32")


accuracy = accuracy_score(y_test, predicted_classes)
print(f"Accuracy: {accuracy:.2f}")


recall = recall_score(y_test, predicted_classes)
print(f"Recall: {recall:.2f}")

recall = precision_score(y_test, predicted_classes)
print(f"Recall: {recall:.2f}")


f1 = f1_score(y_test, predicted_classes)
print(f"F1-Score: {f1:.2f}")
print('-------------------------------')

report = classification_report(y_test, predicted_classes)
print("Classification Report:")
print(report)

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step
Accuracy: 0.97
Recall: 0.96
Recall: 0.99
F1-Score: 0.98
-------------------------------
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       147
           1       0.99      0.96      0.98       163

    accuracy                           0.97       310
   macro avg       0.97      0.97      0.97       310
weighted avg       0.97      0.97      0.97       310

